# 서울시 편의점 위치와 매출의 관계 분석

## 프로젝트 개요
- **목적**: 서울시 행정동별 편의점 매출에 영향을 미치는 요인 분석
- **기간**: 2022년 1분기 ~ 2025년 3분기
- **종속변수**: 행정동별 편의점 분기별 추정매출
- **독립변수**: 유동인구, 상권유형, 편의점 점포수(밀집도)

---

## 1. 환경 설정 및 데이터 로드

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

STORE_PATH = r'../data/processed/stores.csv'
SALES_PATH = r'../data/processed/sales.csv'
POPULATION_PATH = r'../data/processed/population.csv'
DISTRICTS_PATH = r'../data/processed/districts.csv'
# OUTPUT_DIR = r'C:/Users/Administrator/Desktop/자료모음/프로젝트_서울시_편의점_분석'

# STORE_PATH = r'C:/Users/Administrator/Desktop/자료모음/점포/0.서울시_상권분석서비스_점포_행정동_2022-2025_통합.csv'
# SALES_PATH = r'C:/Users/Administrator/Desktop/자료모음/추정매출/0.서울시_상권분석서비스_추정매출_행정동_2022-2025_통합.csv'
# POPULATION_PATH = r'C:/Users/Administrator/Desktop/자료모음/서울시 상권분석서비스(길단위인구-행정동).csv'
# DISTRICTS_PATH = r'C:/Users/Administrator/Desktop/자료모음/서울시 상권분석서비스(영역-상권).csv'
# OUTPUT_DIR = r'C:/Users/Administrator/Desktop/자료모음/프로젝트_서울시_편의점_분석'

store_df = pd.read_csv(STORE_PATH, encoding='utf-8-sig')
sales_df = pd.read_csv(SALES_PATH, encoding='utf-8-sig')
pop_df = pd.read_csv(POPULATION_PATH, encoding='utf-8-sig')
area_df = pd.read_csv(DISTRICTS_PATH, encoding='cp949')
# pop_df = pd.read_csv(POPULATION_PATH, encoding='cp949')
# area_df = pd.read_csv(DISTRICTS_PATH, encoding='cp949')

print(f'점포 데이터: {store_df.shape}')
print(f'매출 데이터: {sales_df.shape}')
print(f'유동인구 데이터: {pop_df.shape}')
print(f'상권영역 데이터: {area_df.shape}')

점포 데이터: (528758, 12)
매출 데이터: (256378, 53)
유동인구 데이터: (11475, 25)
상권영역 데이터: (1650, 11)


## 2. 데이터 기간 검증

In [39]:
print('='*60)
print('데이터 기간 검증 (목표: 2022년 1분기 ~ 2025년 3분기)')
print('='*60)

for name, df in [('점포', store_df), ('매출', sales_df), ('유동인구', pop_df)]:
    print(f"\n[{name} 데이터]")
    print(f"  기간: {df['기준_년분기_코드'].min()} ~ {df['기준_년분기_코드'].max()}")
    print(f"  분기 수: {df['기준_년분기_코드'].nunique()}개")

데이터 기간 검증 (목표: 2022년 1분기 ~ 2025년 3분기)

[점포 데이터]
  기간: 20221 ~ 20253
  분기 수: 15개

[매출 데이터]
  기간: 20221 ~ 20253
  분기 수: 15개

[유동인구 데이터]
  기간: 20191 ~ 20253
  분기 수: 27개


## 3. 데이터 전처리 및 통합

In [40]:
# 편의점 데이터 필터링
cvs_store = store_df[store_df['서비스_업종_코드'] == 'CS300002'].copy()
cvs_sales = sales_df[sales_df['서비스_업종_코드'] == 'CS300002'].copy()
pop_df_filtered = pop_df[pop_df['기준_년분기_코드'] >= 20221].copy()

# 상권유형 매핑
area_type_map = {'A': '골목상권', 'D': '발달상권', 'R': '전통시장', 'U': '관광특구'}
area_df['상권유형'] = area_df['상권_구분_코드'].map(area_type_map)
area_by_dong = area_df.groupby('행정동_코드')['상권유형'].agg(lambda x: x.value_counts().index[0]).reset_index()
area_by_dong.columns = ['행정동_코드', '주요_상권유형']

# 데이터 집계
cvs_store_agg = cvs_store.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {'점포_수': 'sum', '프랜차이즈_점포_수': 'sum', '개업_점포_수': 'sum', '폐업_점포_수': 'sum'}).reset_index()
cvs_sales_agg = cvs_sales.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {'당월_매출_금액': 'sum', '당월_매출_건수': 'sum'}).reset_index()
pop_agg = pop_df_filtered.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {'총_유동인구_수': 'sum'}).reset_index()

# 데이터 통합
merged = cvs_sales_agg.merge(cvs_store_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(pop_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(area_by_dong, on='행정동_코드', how='left')
merged['연도'] = (merged['기준_년분기_코드'] // 10).astype(int)
merged['분기'] = (merged['기준_년분기_코드'] % 10).astype(int)

print(f'편의점 점포: {cvs_store.shape[0]:,}건')
print(f'편의점 매출: {cvs_sales.shape[0]:,}건')
print(f'통합 데이터: {merged.shape[0]:,}건 x {merged.shape[1]}컬럼')

편의점 점포: 6,375건
편의점 매출: 6,097건
통합 데이터: 6,097건 x 13컬럼


In [41]:
merged.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,당월_매출_금액,당월_매출_건수,점포_수,프랜차이즈_점포_수,개업_점포_수,폐업_점포_수,총_유동인구_수,주요_상권유형,연도,분기
0,20221,11110515,청운효자동,"1,202,573,186.00",179263,4,5,0,0,"3,627,519.00",골목상권,2022,1
1,20221,11110530,사직동,"3,761,850,443.00",659962,4,25,0,1,"3,402,653.00",골목상권,2022,1
2,20221,11110540,삼청동,"1,226,802,003.00",138702,1,8,0,0,"821,735.00",발달상권,2022,1
3,20221,11110550,부암동,"607,056,495.00",79978,2,6,1,1,"1,283,546.00",골목상권,2022,1
4,20221,11110560,평창동,"568,959,905.00",72916,3,14,1,0,"823,714.00",골목상권,2022,1


---
## 4. EDA (탐색적 데이터 분석)
### 4.1 기초 통계량

In [42]:
print('='*70)
print('기초 통계량')
print('='*70)
merged[['당월_매출_금액', '점포_수', '총_유동인구_수']].describe()

기초 통계량


,당월_매출_금액,점포_수,총_유동인구_수
count,"6,097.00","6,097.00","6,013.00"
mean,"2,711,090,968.92",6.16,"5,593,015.82"
std,"3,201,446,743.07",5.55,"2,962,356.99"
min,"126,285.00",0.00,"23,239.00"
25%,"947,625,957.00",3.00,"3,527,496.00"
50%,"1,719,118,340.00",5.00,"5,221,346.00"
75%,"3,234,061,370.00",8.00,"6,932,029.00"
max,"35,191,306,523.00",51.00,"22,328,990.00"


### 4.2 결측치 확인

In [43]:
print('결측치 현황:')
null_counts = merged.isnull().sum()
null_counts[null_counts > 0]

결측치 현황:


총_유동인구_수     84
주요_상권유형     339
dtype: int64

### 4.3 상권유형별 분포

In [44]:
print('상권유형별 분포:')
type_dist = merged['주요_상권유형'].value_counts()
for t, cnt in type_dist.items():
    print(f'  {t}: {cnt:,}건 ({cnt/len(merged)*100:.1f}%)')
print(f'  결측: {merged["주요_상권유형"].isnull().sum():,}건')

상권유형별 분포:
  골목상권: 5,182건 (85.0%)
  발달상권: 345건 (5.7%)
  전통시장: 231건 (3.8%)
  결측: 339건


### 4.4 이상치 분석 (IQR)

In [45]:
print('='*70)
print('이상치 분석 (IQR 방법)')
print('='*70)

for col in ['당월_매출_금액', '점포_수', '총_유동인구_수']:
    Q1 = merged[col].quantile(0.25)
    Q3 = merged[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outliers = merged[(merged[col] < lower) | (merged[col] > upper)]
    print(f'\n[{col}]')
    print(f'  정상범위: {max(0,lower):,.0f} ~ {upper:,.0f}')
    print(f'  이상치: {len(outliers):,}건 ({len(outliers)/len(merged)*100:.2f}%)')

이상치 분석 (IQR 방법)

[당월_매출_금액]
  정상범위: 0 ~ 6,663,714,490
  이상치: 512건 (8.40%)

[점포_수]
  정상범위: 0 ~ 16
  이상치: 301건 (4.94%)

[총_유동인구_수]
  정상범위: 0 ~ 12,038,828
  이상치: 180건 (2.95%)


### 4.5 상관관계 분석

In [46]:
corr_cols = ['당월_매출_금액', '점포_수', '총_유동인구_수']
corr_matrix = merged[corr_cols].corr()

print('상관계수 매트릭스:')
print(corr_matrix.round(4))

print('\n해석:')
print(f'  매출 vs 점포수: {corr_matrix.loc["당월_매출_금액", "점포_수"]:.4f} (강한 양의 상관)')
print(f'  매출 vs 유동인구: {corr_matrix.loc["당월_매출_금액", "총_유동인구_수"]:.4f} (중간 양의 상관)')
print(f'  점포수 vs 유동인구: {corr_matrix.loc["점포_수", "총_유동인구_수"]:.4f} (중간 양의 상관)')

상관계수 매트릭스:
          당월_매출_금액  점포_수  총_유동인구_수
당월_매출_금액      1.00  0.81      0.49
점포_수          0.81  1.00      0.48
총_유동인구_수      0.49  0.48      1.00

해석:
  매출 vs 점포수: 0.8065 (강한 양의 상관)
  매출 vs 유동인구: 0.4940 (중간 양의 상관)
  점포수 vs 유동인구: 0.4785 (중간 양의 상관)


### 4.6 연도별 평균 현황

In [47]:
yearly = merged.groupby('연도').agg({
    '당월_매출_금액': 'mean',
    '점포_수': 'mean',
    '총_유동인구_수': 'mean'
}).round(0)

print('연도별 평균 현황:')
for year in yearly.index:
    print(f'\n[{year}년]')
    print(f'  평균 매출: {yearly.loc[year, "당월_매출_금액"]:,.0f}원')
    print(f'  평균 점포수: {yearly.loc[year, "점포_수"]:.2f}개')
    print(f'  평균 유동인구: {yearly.loc[year, "총_유동인구_수"]:,.0f}명')

연도별 평균 현황:

[2022년]
  평균 매출: 2,748,484,308원
  평균 점포수: 6.00개
  평균 유동인구: 5,678,200명

[2023년]
  평균 매출: 2,785,407,949원
  평균 점포수: 6.00개
  평균 유동인구: 5,614,249명

[2024년]
  평균 매출: 2,694,249,705원
  평균 점포수: 6.00개
  평균 유동인구: 5,552,880명

[2025년]
  평균 매출: 2,583,634,058원
  평균 점포수: 6.00개
  평균 유동인구: 5,504,944명


### 4.7 상권유형별 매출 현황

In [48]:
type_stats = merged.groupby('주요_상권유형').agg({
    '당월_매출_금액': ['mean', 'median', 'std'],
    '점포_수': 'mean',
    '총_유동인구_수': 'mean'
}).round(0)

print('상권유형별 매출 현황:')
for t in type_stats.index:
    if pd.notna(t):
        print(f'\n[{t}]')
        print(f'  평균 매출: {type_stats.loc[t, ("당월_매출_금액", "mean")]:,.0f}원')
        print(f'  중앙값 매출: {type_stats.loc[t, ("당월_매출_금액", "median")]:,.0f}원')
        print(f'  평균 점포수: {type_stats.loc[t, ("점포_수", "mean")]:.1f}개')

상권유형별 매출 현황:

[골목상권]
  평균 매출: 2,537,951,651원
  중앙값 매출: 1,686,102,469원
  평균 점포수: 6.0개

[발달상권]
  평균 매출: 6,575,520,394원
  중앙값 매출: 5,518,513,166원
  평균 점포수: 12.0개

[전통시장]
  평균 매출: 2,852,941,173원
  중앙값 매출: 1,955,967,551원
  평균 점포수: 4.0개


In [49]:
#통합 데이터 저장
merged.to_csv('./통합_데이터.csv', encoding='utf-8-sig', index=False)
print(f'통합 데이터 저장 완료: {merged.shape}')

통합 데이터 저장 완료: (6097, 13)


In [50]:
# 기간 검증
print(f"분기 범위: {merged['기준_년분기_코드'].min()} ~ {merged['기준_년분기_코드'].max()}")
print(f"분기 수: {merged['기준_년분기_코드'].nunique()}개")
print(f"분기별 목록: {sorted(merged['기준_년분기_코드'].unique())}")

# 행정동 수 검증
print(f"행정동 수: {merged['행정동_코드'].nunique()}개")

# 분기별 행정동 수 확인
print(merged.groupby('기준_년분기_코드')['행정동_코드'].nunique())

분기 범위: 20221 ~ 20253
분기 수: 15개
분기별 목록: [np.int64(20221), np.int64(20222), np.int64(20223), np.int64(20224), np.int64(20231), np.int64(20232), np.int64(20233), np.int64(20234), np.int64(20241), np.int64(20242), np.int64(20243), np.int64(20244), np.int64(20251), np.int64(20252), np.int64(20253)]
행정동 수: 414개
기준_년분기_코드
20221    409
20222    408
20223    409
20224    409
20231    408
20232    407
20233    407
20234    406
20241    406
20242    405
20243    404
20244    404
20251    404
20252    404
20253    407
Name: 행정동_코드, dtype: int64


In [51]:
# 셀 1
print('=' * 70)
print('통합 데이터 설명')
print('=' * 70)
print('\n[1] 데이터 정의')
print('    서울시 행정동별 편의점 분기별 집계 데이터')
print('    - 분석 단위: 행정동 × 분기')
print(f'    - 총 Row 수: {merged.shape[0]:,}건')
print(f'    - 총 컬럼 수: {merged.shape[1]}개')

통합 데이터 설명

[1] 데이터 정의
    서울시 행정동별 편의점 분기별 집계 데이터
    - 분석 단위: 행정동 × 분기
    - 총 Row 수: 6,097건
    - 총 컬럼 수: 13개


In [52]:
# 셀 2
print('[2] 기간 검증')
print(f'    - 분기 범위: {merged["기준_년분기_코드"].min()} ~ {merged["기준_년분기_코드"].max()}')
print(f'    - 분기 수: {merged["기준_년분기_코드"].nunique()}개')
print(f'    - 분기 목록: {sorted(merged["기준_년분기_코드"].unique())}')

[2] 기간 검증
    - 분기 범위: 20221 ~ 20253
    - 분기 수: 15개
    - 분기 목록: [np.int64(20221), np.int64(20222), np.int64(20223), np.int64(20224), np.int64(20231), np.int64(20232), np.int64(20233), np.int64(20234), np.int64(20241), np.int64(20242), np.int64(20243), np.int64(20244), np.int64(20251), np.int64(20252), np.int64(20253)]


In [53]:
# 셀 3
print('[3] 행정동 검증')
print(f'    - 행정동 수: {merged["행정동_코드"].nunique()}개')
print(f'    - 계산: {merged.shape[0]:,}건 ÷ {merged["기준_년분기_코드"].nunique()}분기 = 약 {merged.shape[0] // merged["기준_년분기_코드"].nunique()}개 행정동/분기')

[3] 행정동 검증
    - 행정동 수: 414개
    - 계산: 6,097건 ÷ 15분기 = 약 406개 행정동/분기


In [54]:
# 셀 4
print('[4] 원본 데이터 → 통합 데이터 변환 과정')
print('    ┌─────────────────────────────────────────────────────────────┐')
print('    │ 원본 데이터                      │ 통합 데이터에 포함된 정보  │')
print('    ├─────────────────────────────────────────────────────────────┤')
print('    │ sales.csv (매출)                 │ 당월_매출_금액, 당월_매출_건수 │')
print('    │ stores.csv (점포)                │ 점포_수, 프랜차이즈_점포_수 등 │')
print('    │ population.csv (유동인구)        │ 총_유동인구_수              │')
print('    │ districts.csv (상권영역)         │ 주요_상권유형               │')
print('    └─────────────────────────────────────────────────────────────┘')

[4] 원본 데이터 → 통합 데이터 변환 과정
    ┌─────────────────────────────────────────────────────────────┐
    │ 원본 데이터                      │ 통합 데이터에 포함된 정보  │
    ├─────────────────────────────────────────────────────────────┤
    │ sales.csv (매출)                 │ 당월_매출_금액, 당월_매출_건수 │
    │ stores.csv (점포)                │ 점포_수, 프랜차이즈_점포_수 등 │
    │ population.csv (유동인구)        │ 총_유동인구_수              │
    │ districts.csv (상권영역)         │ 주요_상권유형               │
    └─────────────────────────────────────────────────────────────┘


In [55]:
# 셀 5
print('[5] 집계 방식')
print('    원본 (수십만 건) → 편의점 필터링 → 행정동+분기별 GROUP BY → 4개 데이터 MERGE')

[5] 집계 방식
    원본 (수십만 건) → 편의점 필터링 → 행정동+분기별 GROUP BY → 4개 데이터 MERGE


In [56]:
# 셀 6
print('[6] 분기별 데이터 건수')
quarterly_counts = merged.groupby('기준_년분기_코드')['행정동_코드'].nunique()
for q, cnt in quarterly_counts.items():
    year = q // 10
    quarter = q % 10
    print(f'    {year}년 {quarter}분기: {cnt}개 행정동')

[6] 분기별 데이터 건수
    2022년 1분기: 409개 행정동
    2022년 2분기: 408개 행정동
    2022년 3분기: 409개 행정동
    2022년 4분기: 409개 행정동
    2023년 1분기: 408개 행정동
    2023년 2분기: 407개 행정동
    2023년 3분기: 407개 행정동
    2023년 4분기: 406개 행정동
    2024년 1분기: 406개 행정동
    2024년 2분기: 405개 행정동
    2024년 3분기: 404개 행정동
    2024년 4분기: 404개 행정동
    2025년 1분기: 404개 행정동
    2025년 2분기: 404개 행정동
    2025년 3분기: 407개 행정동


In [57]:
# 셀 7
print('=' * 70)
print('결론: 통합 데이터 하나로 시각화 및 심층분석 수행 가능')
print('=' * 70)

결론: 통합 데이터 하나로 시각화 및 심층분석 수행 가능
